<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/330_SalesEnab_Nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sales Enablement Orchestrator Nodes

In [ ]:
"""Sales Enablement Orchestrator Nodes

This module contains all nodes for the Sales Enablement Orchestrator workflow.
Following the MVP-first approach: all nodes are rule-based (no LLM) for Phase 1-7.
"""

from typing import Dict, Any
from config import SalesEnablementOrchestratorState


def goal_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Goal Node: Define the goal for the Sales Enablement Orchestrator.

    This is a simple rule-based goal definition that sets the framework.
    No dependencies - this is the entry point.
    """
    lead_id = state.get("lead_id")
    rep_id = state.get("rep_id")
    focus_area = state.get("focus_area")

    # Build focus areas based on input
    if focus_area:
        focus_areas = [focus_area]
    else:
        focus_areas = [
            "lead_prioritization",
            "customer_needs_analysis",
            "outreach_generation",
            "follow_up_coordination",
            "rep_nudging",
            "deal_insights",
            "historical_insights",
            "executive_reporting"
        ]

    goal = {
        "objective": "Enable sales team performance by prioritizing leads, analyzing customer needs, generating outreach, coordinating follow-ups, nudging reps, and surfacing actionable insights",
        "lead_id": lead_id,
        "rep_id": rep_id,
        "focus_areas": focus_areas,
        "scope": "all_leads" if not lead_id else "single_lead",
        "scope_reps": "all_reps" if not rep_id else "single_rep"
    }

    return {
        "goal": goal,
        "errors": state.get("errors", [])
    }


def planning_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan based on goal.

    This creates a step-by-step plan. Rule-based, no LLM needed.
    Only depends on goal_node.
    """
    goal = state.get("goal")

    if not goal:
        return {
            "errors": state.get("errors", []) + ["planning_node: goal is required"]
        }

    focus_areas = goal.get("focus_areas", [])

    # Build plan based on focus areas
    plan = []
    step_num = 1

    # Always start with data loading
    plan.append({
        "step": step_num,
        "name": "data_loading",
        "description": "Load all data files (leads, reps, interactions, deals, signals) and create lookup dictionaries",
        "dependencies": [],
        "outputs": ["leads", "sales_reps", "interactions", "deals", "signals", "leads_lookup", "reps_lookup", "interactions_lookup", "deals_lookup", "signals_lookup"]
    })
    step_num += 1

    # Lead prioritization (if in focus areas)
    if "lead_prioritization" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "lead_prioritization",
            "description": "Rank and prioritize leads based on intent, engagement, risk, and urgency",
            "dependencies": ["data_loading"],
            "outputs": ["prioritized_leads", "top_priority_leads"]
        })
        step_num += 1

    # Customer needs analysis (if in focus areas)
    if "customer_needs_analysis" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "customer_needs_analysis",
            "description": "Analyze customer pain points, buying signals, and objection likelihood",
            "dependencies": ["data_loading"],
            "outputs": ["customer_needs_analysis"]
        })
        step_num += 1

    # Outreach generation (if in focus areas)
    if "outreach_generation" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "outreach_generation",
            "description": "Generate personalized outreach recommendations with message drafts and timing",
            "dependencies": ["lead_prioritization", "customer_needs_analysis"],
            "outputs": ["outreach_recommendations"]
        })
        step_num += 1

    # Follow-up coordination (if in focus areas)
    if "follow_up_coordination" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "follow_up_coordination",
            "description": "Identify and track follow-up actions, detect overdue items",
            "dependencies": ["data_loading"],
            "outputs": ["follow_up_actions"]
        })
        step_num += 1

    # Rep nudging (if in focus areas)
    if "rep_nudging" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "rep_nudging",
            "description": "Generate nudges for sales reps based on overdue follow-ups, stalled deals, and high-priority leads",
            "dependencies": ["follow_up_coordination", "lead_prioritization", "deal_insights"],
            "outputs": ["rep_nudges"]
        })
        step_num += 1

    # Deal insights (if in focus areas)
    if "deal_insights" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "deal_insights",
            "description": "Detect stalled deals, at-risk deals, and opportunities",
            "dependencies": ["data_loading"],
            "outputs": ["deal_insights", "stalled_deals", "at_risk_deals"]
        })
        step_num += 1

    # Historical insights (if in focus areas)
    if "historical_insights" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "historical_insights",
            "description": "Analyze win/loss patterns from past deals to surface actionable insights",
            "dependencies": ["data_loading"],
            "outputs": ["win_patterns", "loss_patterns"]
        })
        step_num += 1

    # Executive reporting (if in focus areas)
    if "executive_reporting" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "executive_reporting",
            "description": "Generate pipeline summary and rep performance metrics",
            "dependencies": ["deal_insights", "lead_prioritization"],
            "outputs": ["pipeline_summary", "rep_performance_summary"]
        })
        step_num += 1

    # Always end with report generation
    plan.append({
        "step": step_num,
        "name": "report_generation",
        "description": "Generate final markdown report with all insights and recommendations",
        "dependencies": [p["name"] for p in plan if p["name"] != "report_generation"],
        "outputs": ["enablement_report", "report_file_path"]
    })

    return {
        "plan": plan,
        "errors": state.get("errors", [])
    }



# Test Phase 1: Foundation Nodes (Goal & Planning)

In [ ]:
"""Test Phase 1: Foundation Nodes (Goal & Planning)

Test goal_node and planning_node independently.
"""

from agents.sales_enablement.nodes import goal_node, planning_node
from config import SalesEnablementOrchestratorState


def test_goal_node():
    """Test goal_node with no input"""
    print("Testing goal_node...")

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = goal_node(state)

    assert "goal" in result, "goal_node should return 'goal'"
    assert result["goal"]["objective"] is not None, "goal should have objective"
    assert len(result["goal"]["focus_areas"]) > 0, "goal should have focus_areas"
    assert result["goal"]["scope"] == "all_leads", "scope should be 'all_leads' when no lead_id"

    print(f"✅ goal_node test passed!")
    print(f"   Objective: {result['goal']['objective'][:60]}...")
    print(f"   Focus areas: {len(result['goal']['focus_areas'])} areas")
    print(f"   Scope: {result['goal']['scope']}")
    print()


def test_goal_node_with_lead_id():
    """Test goal_node with specific lead_id"""
    print("Testing goal_node with lead_id...")

    state: SalesEnablementOrchestratorState = {
        "lead_id": "L-001",
        "errors": []
    }

    result = goal_node(state)

    assert result["goal"]["lead_id"] == "L-001", "goal should include lead_id"
    assert result["goal"]["scope"] == "single_lead", "scope should be 'single_lead'"

    print(f"✅ goal_node with lead_id test passed!")
    print(f"   Lead ID: {result['goal']['lead_id']}")
    print(f"   Scope: {result['goal']['scope']}")
    print()


def test_goal_node_with_focus_area():
    """Test goal_node with specific focus_area"""
    print("Testing goal_node with focus_area...")

    state: SalesEnablementOrchestratorState = {
        "focus_area": "lead_prioritization",
        "errors": []
    }

    result = goal_node(state)

    assert result["goal"]["focus_areas"] == ["lead_prioritization"], "focus_areas should match input"

    print(f"✅ goal_node with focus_area test passed!")
    print(f"   Focus areas: {result['goal']['focus_areas']}")
    print()


def test_planning_node():
    """Test planning_node with goal"""
    print("Testing planning_node...")

    state: SalesEnablementOrchestratorState = {
        "goal": {
            "objective": "Test objective",
            "focus_areas": ["lead_prioritization", "outreach_generation"]
        },
        "errors": []
    }

    result = planning_node(state)

    assert "plan" in result, "planning_node should return 'plan'"
    assert len(result["plan"]) > 0, "plan should have steps"

    # Check that plan includes data_loading (always first)
    assert result["plan"][0]["name"] == "data_loading", "First step should be data_loading"

    # Check that plan includes requested focus areas
    plan_names = [step["name"] for step in result["plan"]]
    assert "lead_prioritization" in plan_names, "plan should include lead_prioritization"
    assert "outreach_generation" in plan_names, "plan should include outreach_generation"

    # Check that plan ends with report_generation
    assert result["plan"][-1]["name"] == "report_generation", "Last step should be report_generation"

    print(f"✅ planning_node test passed!")
    print(f"   Total steps: {len(result['plan'])}")
    print(f"   Steps: {[step['name'] for step in result['plan']]}")
    print()


def test_planning_node_without_goal():
    """Test planning_node error handling when goal is missing"""
    print("Testing planning_node error handling...")

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = planning_node(state)

    assert "errors" in result, "planning_node should return errors"
    assert len(result["errors"]) > 0, "should have error when goal is missing"
    assert "goal is required" in result["errors"][0], "error should mention goal"

    print(f"✅ planning_node error handling test passed!")
    print(f"   Error: {result['errors'][0]}")
    print()


def test_goal_and_planning_chain():
    """Test goal_node → planning_node chain"""
    print("Testing goal_node → planning_node chain...")

    # Start with empty state
    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    # Run goal_node (merge result into state)
    goal_result = goal_node(state)
    state.update(goal_result)
    assert "goal" in state, "goal should be in state after goal_node"

    # Run planning_node (merge result into state)
    plan_result = planning_node(state)
    state.update(plan_result)
    assert "plan" in state, "plan should be in state after planning_node"
    assert len(state["errors"]) == 0, "should have no errors in successful chain"

    print(f"✅ goal_node → planning_node chain test passed!")
    print(f"   Goal objective: {state['goal']['objective'][:50]}...")
    print(f"   Plan steps: {len(state['plan'])}")
    print()


if __name__ == "__main__":
    print("=" * 60)
    print("Phase 1: Foundation Nodes - Test Suite")
    print("=" * 60)
    print()

    test_goal_node()
    test_goal_node_with_lead_id()
    test_goal_node_with_focus_area()
    test_planning_node()
    test_planning_node_without_goal()
    test_goal_and_planning_chain()

    print("=" * 60)
    print("✅ All Phase 1 tests passed!")
    print("=" * 60)



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_007_TEMPLATE copy % python test_sales_enablement_phase1.py
============================================================
Phase 1: Foundation Nodes - Test Suite
============================================================

Testing goal_node...
✅ goal_node test passed!
   Objective: Enable sales team performance by prioritizing leads, analyzi...
   Focus areas: 8 areas
   Scope: all_leads

Testing goal_node with lead_id...
✅ goal_node with lead_id test passed!
   Lead ID: L-001
   Scope: single_lead

Testing goal_node with focus_area...
✅ goal_node with focus_area test passed!
   Focus areas: ['lead_prioritization']

Testing planning_node...
✅ planning_node test passed!
   Total steps: 4
   Steps: ['data_loading', 'lead_prioritization', 'outreach_generation', 'report_generation']

Testing planning_node error handling...
✅ planning_node error handling test passed!
   Error: planning_node: goal is required

Testing goal_node → planning_node chain...
✅ goal_node → planning_node chain test passed!
   Goal objective: Enable sales team performance by prioritizing lead...
   Plan steps: 10

============================================================
✅ All Phase 1 tests passed!
============================================================
